#### 步驟
1. 讀取 地區資訊 和 東方果實蠅的東西
    out_put = di, df
2. 進行 data_organization()
    out_put = X, y, 分級初始值, 級距, 級數
3. 進行 data balancing
    out_put = X, y
3.5 讀取欲預測之天氣()
    Out_put = 欲預測之天氣_X
4. 重複十次以下步驟 : 尚未完成
    input = X, y, 欲預測之天氣, 害蟲名稱

    把data 八二分  計算 score 的值，儲存到buffer_scoring
    用 training data 預測   with   X = 輸入的天氣，儲存到buffer_predict

    svm(kernel = rbf, linear, poly, sigmoid)
    RandomForest(n_estimator = 20, 25, 30, 35)
5. 計算
buffer_scoring 的平均與標準差
如何選取預測值
    出現最多次的標籤
    把 scoring的平方 乘上 buffer_predict 除以 sum(scoring的平方)
    選取 top 7 scoring 的 prediction ，選取出現最多次的標籤
6. 儲存資訊 in list 包括 害蟲名稱 縣市名稱 n天後預測 預測規模 誤差 初始值 級距 級數

In [1]:
import pandas as pd
import time
import os 
import numpy as np
from sklearn.metrics import accuracy_score
from sklearn.decomposition import RandomizedPCA
from sklearn.learning_curve import learning_curve
from bs4 import BeautifulSoup
import datetime

import matplotlib
%matplotlib inline

import matplotlib.pyplot as plt
import csv
from sklearn import svm
import numpy as np
from sklearn import neighbors
from sklearn.cross_validation import train_test_split
from sklearn.preprocessing import StandardScaler
import math
import pylab as pl
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.cross_validation import cross_val_score
import seaborn

print('成功')

成功


In [2]:
def read_station_data():
    with open('.\\data\\地區資訊new.csv', 'r') as f: 
        reader = csv.reader(f)
        station_data = list(list(rec) for rec in csv.reader(f, delimiter=',')) #reads csv into a list of lists
        f.close()
    return station_data
def read_pest_data(pest):
    with open('.\\data\\蔬果重要害蟲防疫旬報_'+pest+'_new.csv','r') as f:
        reader = csv.reader(f)
        pest_data = list(list(rec) for rec in csv.reader(f, delimiter=','))
        f.close()
    return pest_data

In [3]:
def todate():
    import time
    todate_year = time.strftime("%Y")
    todate_month = time.strftime("%m")
    if int(todate_month) <= 9:
        todate_month = todate_month.strip('0') 
    todate_day = time.strftime("%d")
    return todate_year, todate_month, todate_day

def period10d():
    period10d_year = (datetime.datetime.now()-datetime.timedelta(days=10)).strftime("%y")
    period10d_month = (datetime.datetime.now()-datetime.timedelta(days=10)).strftime("%m")
    if int(period10d_month) <= 9:
        period10d_month = period10d_month.strip('0') 
    period10d_day = (datetime.datetime.now()-datetime.timedelta(days=10)).strftime("%d")
    return period10d_year, period10d_month, period10d_day

In [4]:
def open_data(filename, row_sd, n_start, n_multi, n_time):

    print(filename)
    with open(filename,'r',encoding='utf-8', errors='ignore') as f:
        reader = csv.reader(f)
        buffer = list(list(rec) for rec in csv.reader(f, delimiter=',')) #reads csv into a list of lists
        f.close()
    
    X, y = data_organization(buffer, row_sd, n_multi, n_start, n_time)
    return X, y


In [5]:
def data_organization(buffer, row_sd, n_start, n_multi, n_time):
    
    while [] in buffer:
        buffer.remove([])
    input_x=[]
    real_y=[]

    range_level = []
    range_level.append(0)
    for i in range(n_time):
        range_level.append(n_start*pow(n_multi,i))
    range_level[-1]=(10000)
    #print('data_organization_',range_level)

    for buffera in buffer:
        for i in range(n_time):
            if float(buffera[6]) < range_level[i+1] and float(float(buffera[6])) >= range_level[i]:
                buffera[6] = str(i+1)
        input_x.append(buffera[7:])
        real_y.append(float(buffera[6]))
        
    buffer_a = []
    buffer_b = []
    min_length=50
    max_length=30
    for element in input_x:
        if len(element) > max_length:
            max_length = len(element)
        for elementa in element:
            if elementa == '':
                buffer_a.append(0)
            elif elementa == 'X' or elementa =='T':
                buffer_a.append(0)
            else:
                buffer_a.append(float(elementa))
        buffer_b.append(buffer_a)
        buffer_a = []
    for element in buffer_b:
        while len(element) != max_length:
            element.append(0)
            if len(element) > max_length:
                break #just in case
    X = np.array(buffer_b)
    y = np.array(real_y)
    return X, y

In [6]:
def make_data_balanced(X,y):

    uni_class = np.unique(y)
    
    buffer_X =[]
    buffer_y =[]
    
    if len(uni_class) == 2:
        buffer_X,buffer_y = make_data_balanced_2(X,y)
        
    elif len(uni_class) == 1:
        print('天啊，我竟然也遇到這個狀況......那就不處理，直接送出去')
        buffer_X = X
        buffer_y = y
        
    else:
        ratio = ration(y)
        X = X.tolist()
        y = y.tolist()
        buffer_X = X
        buffer_y = y
        count_ratio = 0
        while ratio > 2.1:
            
            count_ratio += 1
            uni_class, uni_class_voting, max_index, min_index, ratio = ration_oo(buffer_y)
            for i in range(len(buffer_y)):
                if y[i] == uni_class[min_index]:
                    for repeat in range(round(ratio)-1):
                        buffer_X.append(X[i]) 
                        buffer_y.append(y[i])
            
            ratio = ration(buffer_y)

    buffer_X = np.array(buffer_X)
    buffer_y = np.array(buffer_y)
    return buffer_X, buffer_y

In [7]:
def make_data_balanced_2(X,y):

    uni_class, uni_class_voting, max_index, min_index, ratio = ration_oo(y)
    if len(uni_class) == 2:
        ratio = ration(y)
        buffer_X = X
        buffer_y = y
        while ratio >= 2.1:
            buffer_X = buffer_X.tolist()
            buffer_y = buffer_y.tolist()
            for i in range(len(y)):
                if y[i] == uni_class[min_index]:
                    for repeat in range(int(ratio)-1):
                        buffer_X.append(X[i])
                        buffer_y.append(y[i])
                else:
                    buffer_X.append(X[i])
                    buffer_y.append(y[i])
            ratio = ration(buffer_y)
        buffer_X = np.array(buffer_X)
        buffer_y = np.array(buffer_y)
        
    else:
        buffer_X = X
        buffer_y = y
    return buffer_X, buffer_y

print('成功')

成功


In [8]:
def ration_oo(y):
    y = np.array(y).astype(int)
    uni_class = np.unique(y)
    uni_class_voting = np.bincount(y)
    
    index = (uni_class_voting == 0)
    buffer = []
    for i in range(len(uni_class_voting)):
        if not index[i]:
            buffer.append(uni_class_voting[i])
    uni_class_voting = np.array(buffer)
    
    max_index = np.argmax(uni_class_voting)
    min_index = np.argmin(uni_class_voting)

    ratio = uni_class_voting[max_index]/uni_class_voting[min_index]
    ratio = int(round(ratio))
    
    return uni_class, uni_class_voting, max_index, min_index, ratio
def ration(y):
    y = np.array(y).astype(int)
    uni_class = np.unique(y)
    uni_class_voting = np.bincount((y))
    
    index = (uni_class_voting == 0)
    buffer = []
    for i in range(len(uni_class_voting)):
        if not index[i]:
            buffer.append(uni_class_voting[i])
    uni_class_voting = np.array(buffer)
    
    max_index = np.argmax(uni_class_voting)
    min_index = np.argmin(uni_class_voting)

    ratio = uni_class_voting[max_index]/uni_class_voting[min_index]
    ratio = int(round(ratio))
    
    return ratio
print('成功')


成功


In [9]:
def predict_X(row_sd):
    sub_buffer=[]
    todate_year, todate_month, todate_day = todate()
    
    if int(todate_day) > 10:
        sub_buffer, need_to_continue =write_day10(row_sd)

    if int(todate_day) < 10 and int(todate_month) != 1:
        sub_buffer, need_to_continue =write_gap_month(row_sd)

    if int(todate_day) < 10 and int(todate_month) == 1:
        sub_buffer, need_to_continue =write_gap_year(row_sd)
    #print(sub_buffer)
    buffera = []
    if need_to_continue == 0:
        for i in range(len(sub_buffer)):
            if sub_buffer[i] == '':
                buffera.append(0)
            elif sub_buffer[i] == 'X':
                buffera.append(0)
            elif sub_buffer[i] == 'T':
                buffera.append(0)
            elif i == 0 or i == 1:
                buffera.append(sub_buffer[i])
            else:
                buffera.append(float(sub_buffer[i]))
                
    return buffera

In [10]:
def write_gap_year(row_sd):
    need_to_continue = 0
    todate_year, todate_month, todate_day = todate()
    period10d_year, period10d_month, period10d_day = period10d()
    row_sd = row_sd[:2]
    
    filename_last_month = '.\\data\\' + row_sd[0] + row_sd[1] + '20'+str(period10d_year) + str(period10d_month) + '.txt' 
    with open(filename_last_month,'r') as f:
        data = f.read()
        f.close()
    sp = BeautifulSoup(data,'lxml')
    tra = sp.find_all('tr')

    try:
        tra.remove(tra[0])
        tra.remove(tra[0])
        tra.remove(tra[0])
    except:
        need_to_continue = 1
        print('tra=[]')
        return row_sd, need_to_continue
    
    traa_count=0
    for traa in tra:
        traa_count = traa_count+1
        if traa_count >= int(period10d_day):
            tda=traa.find_all('td')
            tdaa_count = 0
            for tdaa in tda:
                tdaa_count = tdaa_count +1
                if  tdaa_count == 1:
                    row_sd.append(tdaa.text)
                if  tdaa_count == 8 or tdaa_count==9 or tdaa_count==11 or tdaa_count==18:
                    row_sd.append(tdaa.text.replace('\xa0',''))
                    
    filename = '.\\data\\' + row_sd[0] + row_sd[1] + str(todate_year) + str(todate_month) + '.txt'
    with open(filename,'r') as f:
        data = f.read()
        f.close()
    sp = BeautifulSoup(data,'lxml')
    tra = sp.find_all('tr')
    
    try:
        tra.remove(tra[0])
        tra.remove(tra[0])
        tra.remove(tra[0])
    except:
        need_to_continue = 1
        print('tra=[]')
        return row_sd, need_to_continue
    
    traa_count=0
    for traa in tra:
        traa_count = traa_count+1
        if traa_count < int(todate_day):
            tda=traa.find_all('td')
            tdaa_count = 0
            for tdaa in tda:
                tdaa_count = tdaa_count +1
                if  tdaa_count == 1:
                    row_sd.append(tdaa.text)
                if  tdaa_count == 8 or tdaa_count==9 or tdaa_count==11 or tdaa_count==18:
                    row_sd.append(tdaa.text.replace('\xa0',''))
    
    print('成功')
    return row_sd, need_to_continue

def write_gap_month(row_sd):
    need_to_continue = 0
    todate_year, todate_month, todate_day = todate()
    period10d_year, period10d_month, period10d_day = period10d()
    row_sd = row_sd[:2]
    
    filename_last_month = '.\\data\\' + row_sd[0] + row_sd[1] + '20' + str(period10d_year) + str(period10d_month) + '.txt'
    with open(filename_last_month,'r') as f:
        data = f.read()
        f.close()
    sp = BeautifulSoup(data,'lxml')
    tra = sp.find_all('tr')

    try:
        tra.remove(tra[0])
        tra.remove(tra[0])
        tra.remove(tra[0])
    except:
        need_to_continue = 1
        print('tra=[]')
        return row_sd, need_to_continue
    
    traa_count=0
    for traa in tra:
        traa_count = traa_count+1
        if traa_count >= int(period10d_day):
            tda=traa.find_all('td')
            tdaa_count = 0
            for tdaa in tda:
                tdaa_count = tdaa_count +1
                if  tdaa_count == 1:
                    row_sd.append(tdaa.text)
                if  tdaa_count == 8 or tdaa_count==9 or tdaa_count==11 or tdaa_count==18:
                    row_sd.append(tdaa.text.replace('\xa0',''))
                    
    filename = '.\\data\\' + row_sd[0] + row_sd[1] + str(todate_year) + str(todate_month) + '.txt'
    with open(filename,'r') as f:
        data = f.read()
        f.close()
    sp = BeautifulSoup(data,'lxml')
    tra = sp.find_all('tr')
    tra.remove(tra[0])
    try:
        tra.remove(tra[0])
        tra.remove(tra[0])
        tra.remove(tra[0])
    except:
        need_to_continue = 1
        print('tra=[]')
        return row_sd, need_to_continue
    
    traa_count=0
    for traa in tra:
        traa_count = traa_count+1
        if traa_count < int(todate_day):
            tda=traa.find_all('td')
            tdaa_count = 0
            for tdaa in tda:
                tdaa_count = tdaa_count +1
                if  tdaa_count == 1:
                    row_sd.append(tdaa.text)
                if  tdaa_count == 8 or tdaa_count==9 or tdaa_count==11 or tdaa_count==18:
                    row_sd.append(tdaa.text.replace('\xa0',''))
    
    print('成功')
    return row_sd, need_to_continue



def write_day10(row_sd):
    need_to_continue = 0
    todate_year, todate_month, todate_day = todate()
    filename = '.\\data\\' + row_sd[0] + row_sd[1] + todate_year + todate_month + '.txt' 
    row_sd = row_sd[:2]
    with open(filename,'r') as f:
        data = f.read()
        f.close()
    sp = BeautifulSoup(data,'lxml')
    tra = sp.find_all('tr')
    tra.remove(tra[0])
    try:
        tra.remove(tra[0])
        tra.remove(tra[0])
        tra.remove(tra[0])
    except:
        need_to_return = 1
        print('tra=[]')
        return row_sd, need_to_return
    
    traa_count=0
    for traa in tra:
        traa_count = traa_count+1
        if traa_count < int(todate_day) and traa_count>= int(todate_day)-10 :
            tda=traa.find_all('td')
            tdaa_count = 0
            for tdaa in tda:
                tdaa_count = tdaa_count +1
                if  tdaa_count == 1:
                    row_sd.append(tdaa.text)
                if  tdaa_count == 8 or tdaa_count==9 or tdaa_count==11 or tdaa_count==18:
                    row_sd.append(tdaa.text.replace('\xa0',''))
    print('成功')
    return row_sd, need_to_continue

In [11]:
def predict_weather(X, y, pest_name, predict_X, DF):
    import statistics
    column = ['病蟲害','縣市名稱','時間','規模','準確率','準確率誤差']
    predict_X = np.array(predict_X)

    data = X
    score = []
    predict = []
    prediction = 0
    city_name = ''
    score_standard_deviation = 0
    buffer_X = X
    buffer_predict_X = predict_X
    
    if 'true':
        city_name = predict_X[0] + predict_X[1]
        
        for day in [1, 2, 3]:
            X = buffer_X
            predict_X = buffer_predict_X
            score=[]
            predict=[]
            score_SD = []            
            buffera_X=[]
            for row_X in X:
                buffera_X.append(row_X[:(10-day)*5])
                
            if (len(predict_X) < 35 and day == 3) or (len(predict_X) < 40 and day == 2) or (len(predict_X) < 45 and day == 1):
                print('insufficient data')
                DF2=pd.DataFrame([[pest_name, city_name, day, 0, 'X', 'X']], columns=column)
                DF = DF.append(DF2, ignore_index=True)
                continue
            if len(np.unique(y)) == 1:
                print('insufficient label')
                DF2=pd.DataFrame([[pest_name, city_name, day, y[0], 'all_label', 'are same']], columns=column)
                DF = DF.append(DF2, ignore_index=True)
                continue
            buffera_predict_X = predict_X[-(10-day)*5:]
            X = buffera_X
            predict_X = buffera_predict_X
            
            X = np.array(X)
            predict_X = predict_X.reshape(1,-1)
            for val in range(30,40,1):
                for repeata in range(2):
                    clf = RandomForestClassifier(n_estimators = val)
                    try:
                        X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)
                    except ValueError:
                        print('the number of sample is too less')
                        break
                    except:
                        print('something wrong happen')
                        break
                    
                    if len(y_test) == 1:
                        X = X.reshape(1,-1)
                    
                    y = np.array(y).astype(int)
                    uni_class = np.unique(y)
                    uni_class_voting = np.bincount((y))
                    max_index = np.argmax(uni_class_voting)
                    a = [y_test==max_index]
                    buffer_X_test=[]
                    buffer_y_test=[]
                    for i in range(len(a[0])):
                        if a[0][i]:
                            buffer_X_test.append(X_test[i])
                            buffer_y_test.append(y_test[i])
                    X_test = buffer_X_test
                    y_test = buffer_y_test
                    
                    if X_test == []:
                        print('Found array with 0 feature(s) (shape=(1, 0)) while a minimum of 1 is required.')
                        continue
                    
                    try:
                        score.append(clf.fit(X_train,y_train).score(X_test, y_test))
                        predict.append(clf.fit(X_train,y_train).predict(predict_X))
                    except:
                        print('The number of classes has to be greater than one; got 1')
                        #DF2=pd.DataFrame([[pest_name, city_name, day, y_train[0], 'all_label', 'are same']], columns=column)
                        #DF = DF.append(DF2, ignore_index=True)
                        print(X_test, y_test)
                        pass


            del clf
            for kernel in ['rbf','poly','sigmoid']:
                for C in [1000, 100]:
                    for gamma in [0.01, 0.001]:
                        for i in range(2):
                            try:
                                X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)
                            except ValueError:
                                print('the number of sample is too less')
                                break
                            except:
                                print('something wrong happen')
                                break
                            
                            y = np.array(y).astype(int)
                            uni_class = np.unique(y)
                            uni_class_voting = np.bincount((y))
                            max_index = np.argmax(uni_class_voting)
                            a = [y_test==max_index]
                            buffer_X_test=[]
                            buffer_y_test=[]
                            for i in range(len(a[0])):
                                if a[0][i]:
                                    buffer_X_test.append(X_test[i])
                                    buffer_y_test.append(y_test[i])
                            X_test = buffer_X_test
                            y_test = buffer_y_test
                            
                            clf = SVC(C=C, gamma=gamma, kernel=kernel)
                            
                            if X_test == []:
                                print('Found array with 0 feature(s) (shape=(1, 0)) while a minimum of 1 is required.')
                                continue
                    
                            try:
                                score.append(clf.fit(X_train,y_train).score(X_test, y_test))
                                predict.append(clf.fit(X_train,y_train).predict(predict_X))
                            except ValueError:
                                print('The number of classes has to be greater than one; got 1')
                                #DF2=pd.DataFrame([[pest_name, city_name, day, y_train[0], 'X', 'X']], columns=column)
                                #DF = DF.append(DF2, ignore_index=True)
                                continue_the_loop = 1
                                pass
                            except: 
                                print(X_test, y_test)    

            if (not score) or (not predict) or (len(score) == 1):
                DF2=pd.DataFrame([[pest_name, city_name, day, 0, 'X', 'X']], columns=column)
                DF = DF.append(DF2, ignore_index=True)
                break
            
            weight = [i**2 for i in score]
            prediction = sum([weight[i]*predict[i] for i in range(len(score))])/sum(weight)
            print(predict)
            print(score)
            score_SD = statistics.stdev(score)
            score = statistics.mean(weight)
            DF2=pd.DataFrame([[pest_name, city_name, day, round(prediction[0]), score, score_SD]], columns=column)
            DF = DF.append(DF2, ignore_index=True)
            print(score)
    return DF

In [12]:
column = ['病蟲害','縣市名稱','時間','規模','準確率','準確率誤差']
DF = pd.DataFrame(columns=column)

station_data = read_station_data()
station_data = station_data[1:]
count_sd = 0

for pest_name in ['東方果實蠅','瓜實蠅','甜菜夜蛾','斜紋夜蛾']:
    for row_sd in station_data:
        count_sd = count_sd +1
        filename = ('.\\data\\'+pest_name+ row_sd[0]+ row_sd[1]+'.csv')
        if os.path.isfile(filename):
            #if count_sd < :
            #    continue
            print(datetime.datetime.now())
            X, y = open_data(filename, row_sd, 16, 4, 5) ## output as numpy
            X, y = make_data_balanced(X,y)
            predicta = predict_X(row_sd)
            #print(predicta)
            DF = predict_weather(X, y, pest_name, predicta, DF)
            print(datetime.datetime.now())
            #print(DF)
            print('')
            #if count_sd == 2:
            #    break

2016-09-05 10:39:26.283967
.\data\東方果實蠅台中市石岡區.csv
成功
insufficient data
insufficient data
[array([ 1.]), array([2]), array([2]), array([1]), array([1]), array([2]), array([2]), array([2]), array([2]), array([2]), array([2]), array([2]), array([2]), array([1]), array([2]), array([1]), array([2]), array([1]), array([2]), array([2]), array([1]), array([1]), array([1]), array([1]), array([1]), array([1]), array([1]), array([1]), array([2]), array([2]), array([2]), array([2]), array([2]), array([2]), array([2]), array([2]), array([1]), array([1]), array([1]), array([1]), array([1]), array([1]), array([1]), array([1])]
[1.0, 1.0, 1.0, 0.90000000000000002, 1.0, 0.90909090909090906, 1.0, 0.89473684210526316, 1.0, 1.0, 1.0, 0.91666666666666663, 0.90909090909090906, 1.0, 1.0, 1.0, 1.0, 1.0, 0.80000000000000004, 0.80952380952380953, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 0.88888888888888884, 0.90909090909090906, 0.90476190476190477, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0]

IndexError: list index out of range

In [ ]:
DF_LIST = DF.values.tolist()
import csv
filename = ('.\\data\\20160903outcome.csv')
with open(filename,'w') as f:
    writer = csv.writer(f)
    writer.writerows(DF_LIST)
    f.close
#DF_LIST`

In [13]:
print(DF)

      病蟲害     縣市名稱   時間   規模        準確率      準確率誤差
0   東方果實蠅   台中市石岡區  1.0  0.0          X          X
1   東方果實蠅   台中市石岡區  2.0  0.0          X          X
2   東方果實蠅   台中市石岡區  3.0  1.0   0.950778  0.0528339
3   東方果實蠅   台中市后里區  1.0  0.0          X          X
4   東方果實蠅   台中市后里區  2.0  0.0          X          X
5   東方果實蠅   台中市后里區  3.0  1.0  all_label   are same
6   東方果實蠅   台中市和平區  1.0  0.0          X          X
7   東方果實蠅   台中市和平區  2.0  0.0          X          X
8   東方果實蠅   台中市和平區  3.0  1.0   0.992762  0.0147333
9   東方果實蠅   台中市東勢區  1.0  0.0          X          X
10  東方果實蠅   台中市東勢區  2.0  0.0          X          X
11  東方果實蠅   台中市東勢區  3.0  3.0   0.954545   0.210707
12  東方果實蠅  台東縣台東地區  1.0  0.0          X          X
13  東方果實蠅  台東縣台東地區  2.0  0.0          X          X
14  東方果實蠅  台東縣台東地區  3.0  3.0   0.931818   0.254972
15  東方果實蠅   台南市大內區  1.0  0.0          X          X
16  東方果實蠅   台南市大內區  2.0  0.0          X          X
17  東方果實蠅   台南市大內區  3.0  3.0   0.909091   0.290803
18  東方果實蠅   台南市玉井區  1.0  0.0   

In [15]:
a = DF.values.tolist()


In [16]:
a

[['東方果實蠅', '台中市石岡區', 1.0, 0.0, 'X', 'X'],
 ['東方果實蠅', '台中市石岡區', 2.0, 0.0, 'X', 'X'],
 ['東方果實蠅', '台中市石岡區', 3.0, 1.0, 0.9507776590149531, 0.05283388004962246],
 ['東方果實蠅', '台中市后里區', 1.0, 0.0, 'X', 'X'],
 ['東方果實蠅', '台中市后里區', 2.0, 0.0, 'X', 'X'],
 ['東方果實蠅', '台中市后里區', 3.0, 1.0, 'all_label', 'are same'],
 ['東方果實蠅', '台中市和平區', 1.0, 0.0, 'X', 'X'],
 ['東方果實蠅', '台中市和平區', 2.0, 0.0, 'X', 'X'],
 ['東方果實蠅', '台中市和平區', 3.0, 1.0, 0.9927615265660225, 0.01473326598928107],
 ['東方果實蠅', '台中市東勢區', 1.0, 0.0, 'X', 'X'],
 ['東方果實蠅', '台中市東勢區', 2.0, 0.0, 'X', 'X'],
 ['東方果實蠅', '台中市東勢區', 3.0, 3.0, 0.9545454545454546, 0.21070705494148545],
 ['東方果實蠅', '台東縣台東地區', 1.0, 0.0, 'X', 'X'],
 ['東方果實蠅', '台東縣台東地區', 2.0, 0.0, 'X', 'X'],
 ['東方果實蠅', '台東縣台東地區', 3.0, 3.0, 0.9318181818181818, 0.2549717059293527],
 ['東方果實蠅', '台南市大內區', 1.0, 0.0, 'X', 'X'],
 ['東方果實蠅', '台南市大內區', 2.0, 0.0, 'X', 'X'],
 ['東方果實蠅', '台南市大內區', 3.0, 3.0, 0.9090909090909091, 0.29080336345115265],
 ['東方果實蠅', '台南市玉井區', 1.0, 0.0, 'X', 'X'],
 ['東方果實蠅', '台南市玉井區', 2.0, 0.0,